## Summary


This notebook generates an entire campaign built on the pre-existing template generator function used in the email template editor. The flow has the following steps:

- The user provides a text prompt explaining what the email campaign is for, and how many sends they want to create. They can optionally provide "design guidance", a plain text explanation of what they want the emails to look like, and can upload any number of photos they'd like to include. We'll prompt the user to provide a description of these images which will help the algo figure out where to put them in the templates.

### Building the campaign spec

The first step in creating the campaign is to create the campaign plan, and define campaign styling. The campaign plan is a list of email subject lines and briefs equal to the number of sends that the user has specified they'd like to create for the campaign. This is generated all at once, so that they are cohesive with one another. Campaign styling is a single unified design specification that will be used across the templates in the campaign.

- The prompt is fed to the `seek_clarification` function, which prompts GPT to think of five clarifying questions that might improve the quality of the template recommendations.
- Once the user answers the clarifying questions, the questions and answers are added to all the following prompts
- Next, we run `define_campaign_plan`. This function takes the prompt with clarifications and asks GPT to provide subject lines and briefs for as many email sends as the user specified at the beginning. Each of these will be fed into the email template generator function iteratively.
- Next, we run `define_campaign_styling`. This prompts GPT to specify a design language for all of the emails in the campaign. This design specification is included in the prompt for all email generations, to ensure that the templates that are generated are cohesive.

### Enriching the campaign spec

A major part of our value add here is that we can incorporate images relevant to the customer as well as CTA links to known Momos third party links for inclusion in the email templates. This greatly improves the "completeness" of the resulting email templates.

*Images*
- We have a number of options for integrating images into the templates. As mentioned above, the user can upload images to use in the campaign along with descriptions, which are passed in a JSON to `campaign_images` in the main function. Additionally, we have the parameter `recommend_images`, which takes an array with options `['PEXELS', 'DALLE']`. If these options are specified, we will search for images relevant to the template being generated using the Pexels api (using the `recommend_pexels_photos` function, or will generate them using DALLE (using the `recommend_dalle_photos` function), or both. These images are combined with the images the user uploaded (if present) for consideration by the algorithm when designing templates.

*CTA Links*

- This is currently supplied via a JSON, but we'll need to write a function that queries them from the DB.

### Putting it all together

- Finally, we take all of the email subjects and briefs generated in the campaign plan, with the addition of the clarifications from `seek_clarification`, the design specification from `define_campaign_styling`, and the images from and generate an array of templates

In [2]:
import json
import jsonlines
import math
import openai ## Must be updated to the most recent version
import random
import os
import re
import string
import boto3
from datetime import datetime
import time
import requests

def fetch_key_from_env(env, keys):
    s3_client = boto3.resource('s3')
    env = '.env.' + env

    config = s3_client.Object('prod-be-config', env).get()['Body'].read().decode("utf-8").split('\n')

    keys_to_return = {}
    for i in config:
        row = i.split('=')
        if len(row) > 1:
            row_key = row[0]
            row_value = row[1]
            if row_key in keys:
                keys_to_return.update({row_key : row_value})
    
    return(keys_to_return)

open_ai_key = fetch_key_from_env('production', 'OPEN_AI_GPT3_ACCESS_KEY')
openai.api_key = open_ai_key['OPEN_AI_GPT3_ACCESS_KEY']



In [3]:

## Makes sure we are extracting only the MJML returned by GPT, since it returns some additional
## garbage sometimes

def mjml_checker(text):

    pattern = r"<mjml>(.*?)</mjml>"
    matches = re.findall(pattern, text, re.DOTALL)

    if matches:
        result = matches[0]
        clean_text = result.strip("```")  # Remove surrounding triple backticks if necessary
        clean_text = '<mjml>' + clean_text + '</mjml>'
        return clean_text.replace('\n', '')
    else:
        return None


## In response to the user's prompt, ask a few clarifying questions to improve suggestions    
    
def seek_clarification(prompt, industry):
    
    clarification_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
            {"role": "system", "content": f'''You are a world class marketing expert working for an email marketing firm that is designing email marketing campaigns for {industry} clients. You structure all of your responses in a JSON, with no additional helper text, conversation or comments'''},
            {"role": "user", "content": f'''You've been asked to create an email marketing campaign for the following prompt: {prompt}. Return a JSON with an array named "questions" containing up to 5 clarifying questions you would ask to improve your campaign recommendation.'''}
        ]
        )

    clarification_json = json.loads(clarification_response["choices"][0]['message']['content'])
    
    clarifications = "The client has provided the following answers to clarifying questions:\n\n"
    for question in clarification_json['questions']:
        answer = input(question)
        clarifications += f'''Question: {question}\nAnswer: {answer}\n'''
    
    return(clarifications)



def get_cta_context(file_path = "customer_context.json"):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def get_campaign_images(file_path = "campaign_images.json"):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def define_campaign_plan(prompt = None, clarifications = None, n_designs = 10, industry = "restaurant"):
    
    campaign_plan_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
            {"role": "system", "content": f'''You are a world class marketing expert working for an email marketing firm that is designing email marketing campaigns for {industry} clients.'''},
            {"role": "user", "content": f'''Please describe {n_designs} emails that will be sent as part of a campaign for the following prompt: {prompt}, including a subject line and brief for each email. The brief should be detailed, explaining the content of the email message and providing a description of the layout of the email message. {clarifications}.\n\n The following json contains a list of images that the client would like to use in this campaign with descriptions.  \n\n Remember that all emails in the campaign will be sent sequentially to the target audience, so each email can and should reference previous emails in the campaign. Return a JSON containing an array named emails with {n_designs} items, each containing values for subject_line and email_brief, describing an email in the campaign.'''}
        ]
        )

    # print(campaign_plan_response)
    campaign_plan_json = json.loads(campaign_plan_response["choices"][0]['message']['content'])
    return(campaign_plan_json)


def pick_images(campaign_plan, images, industry):
    
    send_images = []
    
    campaign_plan = campaign_plan["emails"]
    
    for send in campaign_plan:
        
        brief = send['email_brief']
        image_pick = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
            {"role": "system", "content": f'''You are a world class marketing expert working for an email marketing firm that is designing email marketing campaigns for {industry} clients.'''},
            {"role": "user", "content": f'''You are creating an email with the following brief:\n{brief}. Please pick the image best suited to this brief from the following json to be included in the email design. Return the JSON item containing the image url and the image description for your chosen image. If there are no suitable images, return NULL:\n{images}. Return only the json an no additional comments, helper text or clarification'''}
        ]
        )
    
        send_images.append(image_pick["choices"][0]['message']['content'])
        
        
    return(send_images)

def recommend_pexels_photos(campaign_plan, industry):

    def get_pexels_photos(send_images):

        picked_images = []
        picked_images_urls = []
        for image_index in range(0, len(send_images)):
            headers = {
                'Authorization': '563492ad6f91700001000001ca3d7590b4d347c9a309c36c473f0b48',
            }

            params = {
                'query': send_images[image_index]['image_description'],
                'orientation' : send_images[image_index]['orientation'],
                'per_page': '5',
            }

            response = requests.get('https://api.pexels.com/v1/search', params=params, headers=headers)
            response = json.loads(response.content)
            
            for photo_index in range(0,4):
                photos = response['photos'][photo_index]
                if photos['src']['original'] not in picked_images_urls:
                    picked_images.append({"image_description" : photos['alt'], 'image_url' : photos['src']['original']})
                    picked_images_urls.append(photos['src']['original'])
                    break
                else:
                    next

        return(picked_images)


    functions = [
        {
            "name": "search_for_images",
            "description": "Search a stock image library for relevant images",
            "parameters": {
                "type": "object",
                "properties": {
                    "image_description": {
                        "type": "string",
                        "description": "A short description of the desired image (e.g. a slice of pizza). Remember that these must be items that would reasonably contained in a consumer stock images library",
                    },
                    "orientation": {
                        "type": "string",
                        "description": "One of three options: landscape, portrait or square",
                    }
                },
                "required": ["image_description", "orientation"],
            },
        }
    ]


    send_images = []

    image_pick = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": f'''Search for images that would be suitable for an email from a {industry} advertiser with the following characteristics: {send}'''}],
        functions = functions,
        temperature = 1,
        n = 5
    )


    for image in image_pick["choices"]:
        send_images.append(json.loads(image["message"]["function_call"]["arguments"]))

    picked_images = get_pexels_photos(send_images)

    return(picked_images)

def recommend_dalle_photos(image_brief, industry):

    image_request = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
        {"role": "system", "content": f'''You are a world class creative director working on a marketing campaign for an {industry} client. We are designing an email according to the following brief: {image_brief}. Please provide a json of 3 images you'd like to request from our photographer for the email. The JSON should contain an array named `images` containing items with the key `description` with the image description as its value. Return only the JSON with no additional pleasantries, greetings or formatting'''},
        {"role": "user", "content": image_brief}
        ]
    )

    image_descriptions = json.loads(image_request['choices'][0]['message']['content'])

    images = []
    for description in image_descriptions['images']:
        response = openai.Image.create(
            prompt= f'''{description}. Photorealistic, 4K UHD, artistic, soft focus.''',
            n=1,
            size="512x512"
        )
        images.append({'image_description' : description, 'image_url' : response['data'][0]['url']})

    return(images)


def define_campaign_styling(prompt = None, guidance = None, industry = None):
    
    styling_prompt = f'''Please provide a design brief for an email campaign for this prompt: {prompt}.'''
    
    if guidance is not None:
        
        styling_prompt += f''' The client has provided the following notes that should be incorporated into your recommended design system: {guidance}.'''
    
    design_plan_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
            {"role": "system", "content": f'''You are a world class creative director working on a marketing campaign for an {industry} client. Your job is to describe in detail a cohesive design system for an email campaign, including text color, background color, button styling, and common features. The font should always be Roboto. Return only a detailed description of your chosen design system, with no additional comments, clarifications or formatting'''},
            {"role": "user", "content": styling_prompt}
        ]
        )

    design_plan_json = design_plan_response["choices"][0]['message']['content']
    return(design_plan_json)


def create_custom_templates(topic=None, 
                            subject_line=None, 
                            brief=None, 
                            design_recommendation = None, 
                            clarifications = None,
                            brand=None, 
                            bg_color=None, 
                            font_color=None, 
                            cta_context = None, 
                            campaign_images = [], 
                            industry = "restaurant",
                           recommend_images = []):
    
    prompt = f'''Please create an email template for an email with the Topic: {topic}.'''

    if brand is not None:
        prompt += f''' The name of the {industry} client is {brand}.'''

    if bg_color is not None and font_color is not None:
        prompt += f''' The background color of text sections in the email should be {bg_color}. Please use this as 
        the primary color for headers, footers and CTAs. The text color on these sections should be {font_color}. '''

    if subject_line is not None:
        prompt += f''' The subject line of the email is {subject_line}.'''

    if brief is not None:
        prompt += f''' The following is a short brief for the template: {brief}.'''

    if design_recommendation is not None:
        prompt += f''' The email's styling must adhere to the following recommendation from our creative director. Every email must align to this recommendation for cohesion: {design_recommendation}. '''
    
    if clarifications is not None:
        prompt += f''' {clarifications}\n\n'''
        
    if cta_context is not None:
        prompt += f''' The following is a JSON that contains a list of links that can be used for CTAs in the email templates if they match the intent of the email brief, as well as key details about the {industry} client such as the address and phone number:\n\n {cta_context}\n\n'''
    
    if len(recommend_images) > 0:
        
        if 'DALLE' in recommend_images:
            dalle_images = recommend_dalle_photos(f'''{topic}\n{subject_line}\n{brief}''', industry)
            campaign_images.append(dalle_images)
        
        if 'PEXELS' in recommend_images:
            pexels_images = recommend_pexels_photos(f'''{topic}\n{subject_line}\n{brief}''', industry)
            campaign_images.append(pexels_images)
            
    
    prompt += f''' Please ensure that all MJML is completely accurate. Reminder: all opening tags must have closing 
    tags and there can be no spaces between the opening < of a tag and the first word in the tag, or between the 
    closing > of a tag and the last parameter in the tag.''' 
    
    
    if len(campaign_images) > 0:
        prompt += f''' In any place you'd like to put an image, please use an image from the following JSON: {campaign_images} '''
    
    email_templates = []

    template_response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system",
             "content": f'''You are a world class designer working for an email marketing firm that is creating MJML "
                        "email templates for {industry} clients. You are known for splashy designs, that make "
                        "creative use of colors and images to draw in readers. Your designs make use "
                        "of varying font sizes and styling to organize the email layout in an engaging and easy to "
                        "read way. In particular, titles should be large and bolded relative to body text, "
                        "especially in the header of the email, but strictly use Roboto font family. When the user makes a request, you answer only with the MJML code. No additional "
                        "conversation, pleasantries or formatting.'''},
            {"role": "user", "content": prompt}
        ],
        n=1
    )

    email_template = template_response["choices"][0]["message"]["content"]
    clean_response = mjml_checker(email_template)

    return clean_response
        



In [ ]:
def create_email_campaign(brand, industry, design_guidance, prompt, n_designs, campaign_images, recommend_images):
    
    clarifications = seek_clarification(prompt, industry)
    
    campaign_plan = define_campaign_plan(prompt, clarifications, n_designs, industry)
    
    design_recommendation = define_campaign_styling(prompt, design_guidance, industry)
    
    print(design_recommendation)
    
    cta_context = get_cta_context()
    
    email_templates = []
    
    for i in range(0, len(campaign_plan['emails'])):
        
        print(campaign_plan["emails"][i])
        
        template = create_custom_templates(brand = brand, 
                                           topic = prompt, 
                                           subject_line = campaign_plan["emails"][i]['subject_line'], 
                                           brief = campaign_plan["emails"][i]['email_brief'], 
                                           design_recommendation = design_recommendation, 
                                           clarifications = clarifications, 
                                           cta_context = cta_context,
                                           campaign_images =  campaign_images, #picked_images[i],
                                           industry = industry,
                                           recommend_images = recommend_images
                                          )
        
        email_templates.append(template)
        
    return(email_templates)



campaign = create_email_campaign(brand = "Guzman y Gomez", 
                                 industry = "restaurant", 
                                 design_guidance = "Black headers with yellow font, body should have a white background with black font. Use varied font styling and sizes to make it more interesting.", 
                                 prompt = "New Summer Menu", 
                                 n_designs = 5,
                                 campaign_images = get_campaign_images(),
                                recommend_images = ['PEXELS'])


In [334]:
campaign_name = 'summer_menu'

def write_mjml(mjml = None, name = None):
    
    if name is None:
        name = "email_template.mjml"
    
    with open(name, 'w') as file:
        file.write(mjml)

for i in range(0, len(campaign)):
    current_time_unix = int(time.time())
    if campaign[i] is not None:
        write_mjml(campaign[i], f'''./email_templates/{campaign_name}_{current_time_unix}_{i}.mjml''')


In [7]:
define_campaign_plan('Announce New Menu Item', "restaurant")

{'emails': [{'subject_line': 'Introducing Our Delicious New Menu Item!',
   'email_brief': 'This email announces the new menu item to the subscribers. The subject line is attention-grabbing and creates curiosity. The email starts with a mouth-watering image of the new menu item, followed by a catchy headline and a brief description of the dish. It then highlights the unique features and flavors of the dish, along with a call-to-action button inviting customers to learn more.'},
  {'subject_line': 'Discover the Inspiration Behind Our New Menu Item!',
   'email_brief': 'This email builds upon the previous email and focuses on the story behind the creation of the new menu item. It includes an image showcasing the chef preparing the dish, providing a personal touch. The email then delves into the inspiration behind the flavors, ingredients, and culinary techniques used. It also introduces a limited-time offer for subscribers to try the dish at a discounted price.'},
  {'subject_line': 'Cus

In [9]:
industry = 'restaurant'
send = "{'emails': [{'subject_line': 'Introducing Our Delicious New Menu Item!', 'email_brief': 'This email announces the new menu item to the subscribers. The subject line is attention-grabbing and creates curiosity. The email starts with a mouth-watering image of the new menu item, followed by a catchy headline and a brief description of the dish. It then highlights the unique features and flavors of the dish, along with a call-to-action button inviting customers to learn more.'}"

functions = [
        {
            "name": "search_for_images",
            "description": "Search a stock image library for relevant images",
            "parameters": {
                "type": "object",
                "properties": {
                    "image_description": {
                        "type": "string",
                        "description": "A short description of the desired image (e.g. a slice of pizza). Remember that these must be items that would reasonably contained in a consumer stock images library",
                    },
                    "orientation": {
                        "type": "string",
                        "description": "One of three options: landscape, portrait or square",
                    }
                },
                "required": ["image_description", "orientation"],
            },
        }
    ]


send_images = []

image_pick = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[{"role": "user", "content": f'''Search for images that would be suitable for an email from a {industry} advertiser with the following characteristics: {send}'''}],
    functions = functions,
    temperature = 1,
    n = 5
)


In [10]:
image_pick

<OpenAIObject chat.completion id=chatcmpl-7iHzJWVLUJ4qnPK5ybcxvMA7IkDnF at 0x7f969188fae0> JSON: {
  "id": "chatcmpl-7iHzJWVLUJ4qnPK5ybcxvMA7IkDnF",
  "object": "chat.completion",
  "created": 1690789933,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "search_for_images",
          "arguments": "{\n  \"image_description\": \"delicious new menu item\",\n  \"orientation\": \"landscape\"\n}"
        }
      },
      "finish_reason": "function_call"
    },
    {
      "index": 1,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "search_for_images",
          "arguments": "{\n  \"image_description\": \"new menu item\",\n  \"orientation\": \"landscape\"\n}"
        }
      },
      "finish_reason": "function_call"
    },
    {
      "index": 2,
      "message": {
        "role": "ass